# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb1dc497b20>
├── 'a' --> tensor([[ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341]])
└── 'x' --> <FastTreeValue 0x7fb1dc4f7c70>
    └── 'c' --> tensor([[-1.2538, -0.8776,  1.3843, -0.0242],
                        [ 0.5980, -1.7951, -0.4449,  0.0772],
                        [ 0.3038, -0.0385,  0.0639,  0.6546]])

In [4]:
t.a

tensor([[ 0.8304, -0.2322,  1.8142],
        [-0.4266, -0.6264, -0.9341]])

In [5]:
%timeit t.a

63.3 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb1dc497b20>
├── 'a' --> tensor([[-0.8462, -0.9601, -1.1699],
│                   [ 0.1563, -2.3115, -1.2033]])
└── 'x' --> <FastTreeValue 0x7fb1dc4f7c70>
    └── 'c' --> tensor([[-1.2538, -0.8776,  1.3843, -0.0242],
                        [ 0.5980, -1.7951, -0.4449,  0.0772],
                        [ 0.3038, -0.0385,  0.0639,  0.6546]])

In [7]:
%timeit t.a = new_value

63.9 ns ± 0.0339 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8304, -0.2322,  1.8142],
               [-0.4266, -0.6264, -0.9341]]),
    x: Batch(
           c: tensor([[-1.2538, -0.8776,  1.3843, -0.0242],
                      [ 0.5980, -1.7951, -0.4449,  0.0772],
                      [ 0.3038, -0.0385,  0.0639,  0.6546]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8304, -0.2322,  1.8142],
        [-0.4266, -0.6264, -0.9341]])

In [11]:
%timeit b.a

51.7 ns ± 0.0758 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5277, -1.1084, -0.4141],
               [ 0.1018, -0.1248,  1.2147]]),
    x: Batch(
           c: tensor([[-1.2538, -0.8776,  1.3843, -0.0242],
                      [ 0.5980, -1.7951, -0.4449,  0.0772],
                      [ 0.3038, -0.0385,  0.0639,  0.6546]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.107 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 26.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 366 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 916 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb132774ac0>
├── 'a' --> tensor([[[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]],
│           
│                   [[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]],
│           
│                   [[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]],
│           
│                   [[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]],
│           
│                   [[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]],
│           
│                   [[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]],
│           
│                   [[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]],
│           
│                   [[ 0.8304, -0.2322,  1.8142],
│                    [-0.4266, -0.6264, -0.9341]]])
└── 'x' --> <FastTreeValue 0x7fb13273abe0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb138d8d130>
├── 'a' --> tensor([[ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341],
│                   [ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341],
│                   [ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341],
│                   [ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341],
│                   [ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341],
│                   [ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341],
│                   [ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341],
│                   [ 0.8304, -0.2322,  1.8142],
│                   [-0.4266, -0.6264, -0.9341]])
└── 'x' --> <FastTreeValue 0x7fb13902bac0>
    └── 'c' --> tensor([[-1.2538, -0.8776,  1.3843, -0.0242],
                        [ 0.5980, -1.7951, -0.4449,  0.0772],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 40.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.2538, -0.8776,  1.3843, -0.0242],
                       [ 0.5980, -1.7951, -0.4449,  0.0772],
                       [ 0.3038, -0.0385,  0.0639,  0.6546]],
              
                      [[-1.2538, -0.8776,  1.3843, -0.0242],
                       [ 0.5980, -1.7951, -0.4449,  0.0772],
                       [ 0.3038, -0.0385,  0.0639,  0.6546]],
              
                      [[-1.2538, -0.8776,  1.3843, -0.0242],
                       [ 0.5980, -1.7951, -0.4449,  0.0772],
                       [ 0.3038, -0.0385,  0.0639,  0.6546]],
              
                      [[-1.2538, -0.8776,  1.3843, -0.0242],
                       [ 0.5980, -1.7951, -0.4449,  0.0772],
                       [ 0.3038, -0.0385,  0.0639,  0.6546]],
              
                      [[-1.2538, -0.8776,  1.3843, -0.0242],
                       [ 0.5980, -1.7951, -0.4449,  0.0772],
                       [ 0.3038, -0.0385,  0.0639,  0.6546]],

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 424 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.2538, -0.8776,  1.3843, -0.0242],
                      [ 0.5980, -1.7951, -0.4449,  0.0772],
                      [ 0.3038, -0.0385,  0.0639,  0.6546],
                      [-1.2538, -0.8776,  1.3843, -0.0242],
                      [ 0.5980, -1.7951, -0.4449,  0.0772],
                      [ 0.3038, -0.0385,  0.0639,  0.6546],
                      [-1.2538, -0.8776,  1.3843, -0.0242],
                      [ 0.5980, -1.7951, -0.4449,  0.0772],
                      [ 0.3038, -0.0385,  0.0639,  0.6546],
                      [-1.2538, -0.8776,  1.3843, -0.0242],
                      [ 0.5980, -1.7951, -0.4449,  0.0772],
                      [ 0.3038, -0.0385,  0.0639,  0.6546],
                      [-1.2538, -0.8776,  1.3843, -0.0242],
                      [ 0.5980, -1.7951, -0.4449,  0.0772],
                      [ 0.3038, -0.0385,  0.0639,  0.6546],
                      [-1.2538, -0.8776,  1.3843, -0.0242],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 537 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
